In [7]:
from pathlib import Path
import pandas as pd
from chan import ChanKLineProcessor, validate_processing

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_RAW = ROOT / "data"
DATA_CHAN = ROOT / "chan_data"
DATA_CHAN.mkdir(parents=True, exist_ok=True)

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m", "15m", "30m", "1h", "4h"]

def generate_chan_k(symbol: str, tf: str):
    src = DATA_RAW / symbol / f"{tf}.csv"
    if not src.exists():
        print(f"[skip] missing {src}")
        return

    df = pd.read_csv(src, parse_dates=["timestamp"])
    if df.empty:
        print(f"[skip] empty file {src}")
        return

    print(f"[load] {symbol} {tf} rows={len(df)}")

    try:
        processor = ChanKLineProcessor()
        k = processor.process(df)
        stats = validate_processing(df, k)
    except Exception as e:
        print(f"[error] {symbol} {tf}: {e}")
        return

    out_dir = DATA_CHAN / symbol
    out_dir.mkdir(exist_ok=True, parents=True)
    out_path = out_dir / f"{tf}_chanK.csv"
    k.to_csv(out_path, index=False)

    ratio = len(k) / len(df)
    print(f"[saved] {symbol} {tf}: {len(k)} / {len(df)} ({ratio:.2%})")
    print(f"    ├─ volume_ok={stats['volume_preserved']}, price_ok={stats['price_integrity']}, ts_ok={stats['timestamp_monotonic']}")
    print(f"    └─ reduction={stats['reduction_ratio']:.2%}")

for s in SYMBOLS:
    for tf in TFS:
        generate_chan_k(s, tf)


[load] BTCUSDT 5m rows=57600
[saved] BTCUSDT 5m: 42049 / 57600 (73.00%)
    ├─ volume_ok=True, price_ok=True, ts_ok=True
    └─ reduction=27.00%
[load] BTCUSDT 15m rows=19200
[saved] BTCUSDT 15m: 14227 / 19200 (74.10%)
    ├─ volume_ok=True, price_ok=True, ts_ok=True
    └─ reduction=25.90%
[load] BTCUSDT 30m rows=9600
[saved] BTCUSDT 30m: 7162 / 9600 (74.60%)
    ├─ volume_ok=True, price_ok=True, ts_ok=True
    └─ reduction=25.40%
[load] BTCUSDT 1h rows=4800
[saved] BTCUSDT 1h: 3563 / 4800 (74.23%)
    ├─ volume_ok=True, price_ok=True, ts_ok=True
    └─ reduction=25.77%
[load] BTCUSDT 4h rows=1200
[saved] BTCUSDT 4h: 834 / 1200 (69.50%)
    ├─ volume_ok=True, price_ok=True, ts_ok=True
    └─ reduction=30.50%
[load] ETHUSDT 5m rows=57600
[saved] ETHUSDT 5m: 42981 / 57600 (74.62%)
    ├─ volume_ok=True, price_ok=True, ts_ok=True
    └─ reduction=25.38%
[load] ETHUSDT 15m rows=19200
[saved] ETHUSDT 15m: 14235 / 19200 (74.14%)
    ├─ volume_ok=True, price_ok=True, ts_ok=True
    └─ reduct